Generative Adversarial Networks (GAN) were introduced by Ian Goodfellow in 2014 and attacks the problem of unsupervised learning by training two deep networks, called a Generator and Discriminator. The networks compete and cooperate with each other and in the course of training, both networks eventually learn how to perform their respective tasks.

GAN is almost always explained like the case of a counterfeiter (Generative) and the police (Discriminator). Initially, the counterfeiter will show the police a fake money. The police says it is fake. The police gives feedback to the counterfeiter why the money is fake. The counterfeiter attempts to make a new fake money based on the feedback it received. The police says the money is still fake and offers a new set of feedback. The counterfeiter attempts to make a new fake money based on the latest feedback. The cycle continues indefinitely until the police is fooled by the fake money because it looks real.

A Deep Convolutional GAN (DCGAN) is a model that is able to learn by itself how to synthesize new images. Here we create a simple, proof of concept DCGAN using the MNIST dataset to learn how to write handwritten digits.

In [1]:
# imports
import numpy as np
from keras.datasets import mnist
from matplotlib import pyplot as plt
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Flatten, BatchNormalization, Activation, Reshape, LeakyReLU
from keras.layers.convolutional import Conv2D, Conv2DTranspose, UpSampling2D
from keras.optimizers import RMSprop

Using TensorFlow backend.


In [2]:
# set random seed
seed = 7
np.random.seed(seed)

In [3]:
# define model parameters
depth = 16
depth2 = 64
dim = 7
dropout = 0.4

# Discriminator
A discriminator that tells how real an image is. The sigmoid output is a scalar value of the probability of how real the image is - 0.0 is certainly fake, 1.0 is certainly real, anything in between is a gray area. The difference between a discriminator and a typical CNN is the absence of max-pooling in between layers. Instead, a strided convolution is used for downsampling.

In [4]:
# define the discriminator model
img_rows = 28
img_cols = 28
img_channels = 1
def police(depth, dropout):
    # in: 28 x 28 x 1
    # out: 14 x 14 x 1
    inputs = Input(shape=(img_rows,img_cols,img_channels))
    X = Conv2D(depth*1, 5, strides=2, padding='same')(inputs)
    X = LeakyReLU(alpha=0.2)(X)
    X = Dropout(dropout)(X)
    
    X = Conv2D(depth*2, 5, strides=2, padding='same')(X)
    X = LeakyReLU(alpha=0.2)(X)
    X = Dropout(dropout)(X)
    
    X = Conv2D(depth*4, 5, strides=2, padding='same')(X)
    X = LeakyReLU(alpha=0.2)(X)
    X = Dropout(dropout)(X)
    
    X = Conv2D(depth*8, 5, strides=1, padding='same')(X)
    X = LeakyReLU(alpha=0.2)(X)
    X = Dropout(dropout)(X)
    
    X = Flatten()(X)    
    output = Dense(1, activation='sigmoid')(X)
    model = Model(inputs=inputs, outputs=output)
    return model

In [5]:
# create the discriminator model
discriminator = police(depth, dropout)
optimizer = RMSprop(lr=0.0002, decay=6e-8)
discriminator.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['acc'])
print("Discriminator Model")
print("------------------------")
discriminator.summary()

Discriminator Model
------------------------
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 16)        416       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 32)          12832     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 7, 7, 32)          0         
_________________________________________________________________
dropout_2 (Dropout)          (N

# Generator
The generator synthesizes fake images. The fake images are generated from a 100-dimensional noise (uniform distribution between -1.0 to 1.0) using the inverse of a convolution, called a transposed convolution. Upsampling layers are used instead of  fractionally-strided convolutions because they synthesize more realistic handwriting images. In between layers, batch normalization is used to stabilize learning. The output of the sigmoid at the last layer produces the fake image. 

In [6]:
# define the generator model
def thief(dim, depth2, dropout):
    inputs = Input(shape=(100,))
    # in: 100
    # out: dim x dim x depth*4
    X = Dense(dim*dim*depth2)(inputs)
    X = BatchNormalization(momentum=0.9)(X)
    X = Activation('relu')(X)
    X = Reshape((dim, dim, depth2))(X)
    X = Dropout(dropout)(X)
    
    # in: dim x dim x depth*4
    # out: dim*2 x dim*2 x depth*4 / 2
    X = UpSampling2D()(X)
    X = Conv2DTranspose(int(depth2/2), 5, padding='same')(X)
    X = BatchNormalization(momentum=0.9)(X)
    X = Activation('relu')(X)
    
    X = UpSampling2D()(X)
    X = Conv2DTranspose(int(depth2/4), 5, padding='same')(X)
    X = BatchNormalization(momentum=0.9)(X)
    X = Activation('relu')(X)
    
    X = Conv2DTranspose(int(depth2/8), 5, padding='same')(X)
    X = BatchNormalization(momentum=0.9)(X)
    X = Activation('relu')(X)
    
    # output is a 28 x 28 x 1 grayscale image at [0.0,1.0] per pixel
    X = Conv2DTranspose(1, 5, padding='same')(X)
    output = Activation('sigmoid')(X)
    model = Model(inputs=inputs, outputs=output)
    return model

In [7]:
# define the generator model
generator = thief(dim, depth2*4, dropout)
generator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 12544)             1266944   
_________________________________________________________________
batch_normalization_1 (Batch (None, 12544)             50176     
_________________________________________________________________
activation_1 (Activation)    (None, 12544)             0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 256)         0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 7, 7, 256)         0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 14, 14, 256)       0         
__________

## Adversarial Model
The adversarial model is just the generator-discriminator stacked together. The Generator part is trying to fool the Discriminator and learning from its feedback at the same time. 
<img src="images/adversarial_model.png" style="">

In [8]:
# create the adversarial model
adv_model = Sequential()
adv_model.add(generator)
adv_model.add(discriminator)
AMoptimizer = RMSprop(lr=0.0001, decay=3e-8)
adv_model.compile(optimizer=AMoptimizer, loss='binary_crossentropy', metrics=['acc'])
print("Adversarial Model")
print("------------------------")
adv_model.summary()

Adversarial Model
------------------------
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_2 (Model)              (None, 28, 28, 1)         2394241   
_________________________________________________________________
model_1 (Model)              (None, 1)                 271489    
Total params: 2,665,730
Trainable params: 2,640,194
Non-trainable params: 25,536
_________________________________________________________________


In [9]:
# create the ground truth training set
from tensorflow.examples.tutorials.mnist import input_data
x_train = input_data.read_data_sets("mnist", one_hot=True).train.images
x_train = x_train.reshape(-1, img_rows, img_cols, img_channels).astype(np.float32)

Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz


In [10]:
# validate mnist training input
print(x_train.shape)
print(x_train[0])

(55000, 28, 28, 1)
[[[0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]]

 [[0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]]

 [[0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  

## Training the Models
We first determine if Discriminator model is correct by training it alone with real and fake images. Afterwards, the Discriminator and Adversarial models are trained one after the other.
<img src="images/discriminator_training.png" style="">

In [15]:
# create function to save and plot image outputs
def plot_images(save2file=False, fake=True, samples=16, noise=None, step=0):
    filename = 'mnist.png'
    if fake:
        if noise is None:
            noise = np.random.uniform(-1.0, 1.0, size=[samples, 100])
        else:
            filename = "mnist_%d.png" % step
        images = generator.predict(noise)
    else:
        i = np.random.randint(0, x_train.shape[0], samples)
        images = x_train[i, :, :, :]
    
    plt.figure(figsize=(10,10))
    for i in range(images.shape[0]):
        plt.subplot(4, 4, i+1)
        image = images[i, :, :, :]
        image = np.reshape(image, [img_rows, img_cols])
        plt.imshow(image, cmap='gray')
        plt.axis('off')
    plt.tight_layout()
    if save2file:
        plt.savefig(filename)
        plt.close('all')
    else:
        plt.show()

In [16]:
# define the training function
def train(train_steps, batch_size, save_interval=0):
    noise_input = None
    if save_interval > 0:
        noise_input = np.random.uniform(-1.0, 1.0, size=[16,100])
    for i in range(train_steps):
        # define the truth training image set
        images_train = x_train[np.random.randint(0, x_train.shape[0], size=batch_size), :, :, :]
        noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 100])
        images_fake = generator.predict(noise)
        # create combined image group for training the discriminator
        x = np.concatenate((images_train, images_fake))
        y = np.ones([batch_size*2, 1])
        y[batch_size:, :] = 0
        # train the discriminator model on the image group
        d_loss = discriminator.train_on_batch(x, y)
        
        # train the adversarial model
        noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 100])
        y = np.ones([batch_size, 1])
        a_loss = adv_model.train_on_batch(noise, y)
        
        # print outputs
        mesg = "%d: [Discriminator loss: %f, acc: %f]" % (i, d_loss[0], d_loss[1])
        mesg = ("%s: [Adversarial loss: %f, acc: %f]" % (mesg, a_loss[0], a_loss[1]))
        print(mesg)
        
        # save output samples to file
        if save_interval > 0:
            if (i + 1) % save_interval == 0:
                plot_images(save2file=True, samples=noise_input.shape[0], noise=noise_input, step=(i+1))    

In [14]:
# execute training function and plot images
train(train_steps=5000, batch_size=256, save_interval=250)

0: [Discriminator loss: 0.694369, acc: 0.466797]: [Adversarial loss: 0.707928, acc: 0.097656]
1: [Discriminator loss: 0.678317, acc: 0.607422]: [Adversarial loss: 0.685899, acc: 0.726562]
2: [Discriminator loss: 0.661741, acc: 0.562500]: [Adversarial loss: 0.656999, acc: 0.980469]
3: [Discriminator loss: 0.639216, acc: 0.519531]: [Adversarial loss: 0.611857, acc: 1.000000]
4: [Discriminator loss: 0.608894, acc: 0.539062]: [Adversarial loss: 0.529676, acc: 1.000000]
5: [Discriminator loss: 0.569575, acc: 0.513672]: [Adversarial loss: 0.416652, acc: 1.000000]
6: [Discriminator loss: 0.531489, acc: 0.500000]: [Adversarial loss: 0.282076, acc: 1.000000]
7: [Discriminator loss: 0.493789, acc: 0.500000]: [Adversarial loss: 0.164512, acc: 1.000000]
8: [Discriminator loss: 0.459084, acc: 0.500000]: [Adversarial loss: 0.084239, acc: 1.000000]
9: [Discriminator loss: 0.434782, acc: 0.500000]: [Adversarial loss: 0.039957, acc: 1.000000]
10: [Discriminator loss: 0.410518, acc: 0.500000]: [Adversar

87: [Discriminator loss: 0.673967, acc: 0.697266]: [Adversarial loss: 1.088176, acc: 0.496094]
88: [Discriminator loss: 1.232334, acc: 0.527344]: [Adversarial loss: 4.006560, acc: 0.027344]
89: [Discriminator loss: 0.703142, acc: 0.693359]: [Adversarial loss: 1.285188, acc: 0.445312]
90: [Discriminator loss: 1.125520, acc: 0.564453]: [Adversarial loss: 3.550196, acc: 0.058594]
91: [Discriminator loss: 0.758136, acc: 0.628906]: [Adversarial loss: 1.571297, acc: 0.289062]
92: [Discriminator loss: 0.998718, acc: 0.607422]: [Adversarial loss: 3.324606, acc: 0.062500]
93: [Discriminator loss: 0.748603, acc: 0.615234]: [Adversarial loss: 1.563198, acc: 0.300781]
94: [Discriminator loss: 1.032664, acc: 0.550781]: [Adversarial loss: 3.445504, acc: 0.042969]
95: [Discriminator loss: 0.772264, acc: 0.642578]: [Adversarial loss: 1.209668, acc: 0.464844]
96: [Discriminator loss: 1.146157, acc: 0.546875]: [Adversarial loss: 3.460610, acc: 0.046875]
97: [Discriminator loss: 0.741058, acc: 0.652344]:

173: [Discriminator loss: 0.746329, acc: 0.511719]: [Adversarial loss: 1.034391, acc: 0.175781]
174: [Discriminator loss: 0.729355, acc: 0.517578]: [Adversarial loss: 0.802863, acc: 0.398438]
175: [Discriminator loss: 0.757181, acc: 0.498047]: [Adversarial loss: 1.098691, acc: 0.136719]
176: [Discriminator loss: 0.709835, acc: 0.554688]: [Adversarial loss: 0.675061, acc: 0.562500]
177: [Discriminator loss: 0.779552, acc: 0.478516]: [Adversarial loss: 1.200363, acc: 0.082031]
178: [Discriminator loss: 0.723733, acc: 0.525391]: [Adversarial loss: 0.636245, acc: 0.656250]
179: [Discriminator loss: 0.738284, acc: 0.523438]: [Adversarial loss: 1.112444, acc: 0.097656]
180: [Discriminator loss: 0.672958, acc: 0.578125]: [Adversarial loss: 0.743133, acc: 0.441406]
181: [Discriminator loss: 0.725139, acc: 0.511719]: [Adversarial loss: 0.910514, acc: 0.253906]
182: [Discriminator loss: 0.709366, acc: 0.529297]: [Adversarial loss: 0.812630, acc: 0.351562]
183: [Discriminator loss: 0.736847, acc:

259: [Discriminator loss: 0.698593, acc: 0.501953]: [Adversarial loss: 0.773414, acc: 0.304688]
260: [Discriminator loss: 0.692729, acc: 0.541016]: [Adversarial loss: 0.836588, acc: 0.132812]
261: [Discriminator loss: 0.686743, acc: 0.531250]: [Adversarial loss: 0.721962, acc: 0.417969]
262: [Discriminator loss: 0.697474, acc: 0.537109]: [Adversarial loss: 0.816892, acc: 0.132812]
263: [Discriminator loss: 0.691329, acc: 0.490234]: [Adversarial loss: 0.735533, acc: 0.410156]
264: [Discriminator loss: 0.690288, acc: 0.535156]: [Adversarial loss: 0.840925, acc: 0.164062]
265: [Discriminator loss: 0.685732, acc: 0.562500]: [Adversarial loss: 0.752229, acc: 0.312500]
266: [Discriminator loss: 0.690117, acc: 0.527344]: [Adversarial loss: 0.765682, acc: 0.269531]
267: [Discriminator loss: 0.688768, acc: 0.535156]: [Adversarial loss: 0.770304, acc: 0.292969]
268: [Discriminator loss: 0.693958, acc: 0.517578]: [Adversarial loss: 0.780814, acc: 0.261719]
269: [Discriminator loss: 0.689144, acc:

345: [Discriminator loss: 0.694788, acc: 0.501953]: [Adversarial loss: 0.713147, acc: 0.421875]
346: [Discriminator loss: 0.694287, acc: 0.515625]: [Adversarial loss: 0.752790, acc: 0.261719]
347: [Discriminator loss: 0.692067, acc: 0.525391]: [Adversarial loss: 0.730752, acc: 0.339844]
348: [Discriminator loss: 0.689947, acc: 0.515625]: [Adversarial loss: 0.704590, acc: 0.445312]
349: [Discriminator loss: 0.691647, acc: 0.527344]: [Adversarial loss: 0.748750, acc: 0.339844]
350: [Discriminator loss: 0.685888, acc: 0.572266]: [Adversarial loss: 0.727913, acc: 0.414062]
351: [Discriminator loss: 0.692816, acc: 0.523438]: [Adversarial loss: 0.743620, acc: 0.316406]
352: [Discriminator loss: 0.686000, acc: 0.542969]: [Adversarial loss: 0.702004, acc: 0.464844]
353: [Discriminator loss: 0.687857, acc: 0.544922]: [Adversarial loss: 0.711152, acc: 0.406250]
354: [Discriminator loss: 0.691101, acc: 0.511719]: [Adversarial loss: 0.715528, acc: 0.402344]
355: [Discriminator loss: 0.688786, acc:

...

4603: [Discriminator loss: 0.691971, acc: 0.529297]: [Adversarial loss: 0.682407, acc: 0.558594]
4604: [Discriminator loss: 0.694100, acc: 0.544922]: [Adversarial loss: 0.703635, acc: 0.464844]
4605: [Discriminator loss: 0.698888, acc: 0.494141]: [Adversarial loss: 0.690159, acc: 0.511719]
4606: [Discriminator loss: 0.700182, acc: 0.482422]: [Adversarial loss: 0.696632, acc: 0.492188]
4607: [Discriminator loss: 0.691056, acc: 0.537109]: [Adversarial loss: 0.688583, acc: 0.527344]
4608: [Discriminator loss: 0.698301, acc: 0.496094]: [Adversarial loss: 0.698872, acc: 0.460938]
4609: [Discriminator loss: 0.694187, acc: 0.500000]: [Adversarial loss: 0.690411, acc: 0.515625]
4610: [Discriminator loss: 0.699623, acc: 0.503906]: [Adversarial loss: 0.704415, acc: 0.453125]
4611: [Discriminator loss: 0.701403, acc: 0.478516]: [Adversarial loss: 0.686056, acc: 0.503906]
4612: [Discriminator loss: 0.692108, acc: 0.527344]: [Adversarial loss: 0.694484, acc: 0.476562]
4613: [Discriminator loss: 0.6

4688: [Discriminator loss: 0.694905, acc: 0.515625]: [Adversarial loss: 0.688599, acc: 0.500000]
4689: [Discriminator loss: 0.699987, acc: 0.484375]: [Adversarial loss: 0.695725, acc: 0.464844]
4690: [Discriminator loss: 0.702916, acc: 0.470703]: [Adversarial loss: 0.692737, acc: 0.500000]
4691: [Discriminator loss: 0.694669, acc: 0.501953]: [Adversarial loss: 0.684970, acc: 0.554688]
4692: [Discriminator loss: 0.694285, acc: 0.533203]: [Adversarial loss: 0.697172, acc: 0.496094]
4693: [Discriminator loss: 0.694293, acc: 0.513672]: [Adversarial loss: 0.692902, acc: 0.511719]
4694: [Discriminator loss: 0.707847, acc: 0.468750]: [Adversarial loss: 0.690419, acc: 0.507812]
4695: [Discriminator loss: 0.704256, acc: 0.472656]: [Adversarial loss: 0.689727, acc: 0.484375]
4696: [Discriminator loss: 0.702857, acc: 0.474609]: [Adversarial loss: 0.688321, acc: 0.511719]
4697: [Discriminator loss: 0.702390, acc: 0.482422]: [Adversarial loss: 0.692947, acc: 0.515625]
4698: [Discriminator loss: 0.7

4773: [Discriminator loss: 0.686435, acc: 0.542969]: [Adversarial loss: 0.669393, acc: 0.601562]
4774: [Discriminator loss: 0.705097, acc: 0.494141]: [Adversarial loss: 0.714713, acc: 0.363281]
4775: [Discriminator loss: 0.697568, acc: 0.498047]: [Adversarial loss: 0.694066, acc: 0.488281]
4776: [Discriminator loss: 0.702529, acc: 0.500000]: [Adversarial loss: 0.703256, acc: 0.429688]
4777: [Discriminator loss: 0.702271, acc: 0.482422]: [Adversarial loss: 0.703698, acc: 0.437500]
4778: [Discriminator loss: 0.702782, acc: 0.501953]: [Adversarial loss: 0.710797, acc: 0.375000]
4779: [Discriminator loss: 0.703426, acc: 0.455078]: [Adversarial loss: 0.709625, acc: 0.410156]
4780: [Discriminator loss: 0.698493, acc: 0.484375]: [Adversarial loss: 0.691987, acc: 0.488281]
4781: [Discriminator loss: 0.702496, acc: 0.492188]: [Adversarial loss: 0.690378, acc: 0.527344]
4782: [Discriminator loss: 0.696459, acc: 0.515625]: [Adversarial loss: 0.692542, acc: 0.484375]
4783: [Discriminator loss: 0.6

4858: [Discriminator loss: 0.699515, acc: 0.449219]: [Adversarial loss: 0.693828, acc: 0.488281]
4859: [Discriminator loss: 0.703099, acc: 0.460938]: [Adversarial loss: 0.702202, acc: 0.496094]
4860: [Discriminator loss: 0.697176, acc: 0.521484]: [Adversarial loss: 0.694048, acc: 0.468750]
4861: [Discriminator loss: 0.695678, acc: 0.515625]: [Adversarial loss: 0.690596, acc: 0.492188]
4862: [Discriminator loss: 0.699451, acc: 0.500000]: [Adversarial loss: 0.691199, acc: 0.464844]
4863: [Discriminator loss: 0.698297, acc: 0.515625]: [Adversarial loss: 0.691006, acc: 0.531250]
4864: [Discriminator loss: 0.699201, acc: 0.496094]: [Adversarial loss: 0.701472, acc: 0.476562]
4865: [Discriminator loss: 0.692193, acc: 0.515625]: [Adversarial loss: 0.698965, acc: 0.460938]
4866: [Discriminator loss: 0.698231, acc: 0.525391]: [Adversarial loss: 0.709858, acc: 0.363281]
4867: [Discriminator loss: 0.691436, acc: 0.533203]: [Adversarial loss: 0.706777, acc: 0.425781]
4868: [Discriminator loss: 0.6

4943: [Discriminator loss: 0.701282, acc: 0.472656]: [Adversarial loss: 0.711944, acc: 0.367188]
4944: [Discriminator loss: 0.699436, acc: 0.488281]: [Adversarial loss: 0.712043, acc: 0.398438]
4945: [Discriminator loss: 0.701842, acc: 0.513672]: [Adversarial loss: 0.709668, acc: 0.386719]
4946: [Discriminator loss: 0.690627, acc: 0.527344]: [Adversarial loss: 0.699384, acc: 0.500000]
4947: [Discriminator loss: 0.704243, acc: 0.482422]: [Adversarial loss: 0.699056, acc: 0.464844]
4948: [Discriminator loss: 0.708691, acc: 0.437500]: [Adversarial loss: 0.713555, acc: 0.375000]
4949: [Discriminator loss: 0.696814, acc: 0.503906]: [Adversarial loss: 0.693588, acc: 0.500000]
4950: [Discriminator loss: 0.690489, acc: 0.558594]: [Adversarial loss: 0.693903, acc: 0.507812]
4951: [Discriminator loss: 0.690676, acc: 0.541016]: [Adversarial loss: 0.702089, acc: 0.449219]
4952: [Discriminator loss: 0.694511, acc: 0.537109]: [Adversarial loss: 0.711058, acc: 0.414062]
4953: [Discriminator loss: 0.6